In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import f1_score


from torch import tensor

from shutil import copyfile

copyfile(src = "../input/generateswords/generate_data.py", dst = "../working/generate_data.py")

from generate_data import generate_data, data_loader

# onehot = pd.Series(data = [tensor([1,0,0]),tensor([0,1,0]),tensor([0,0,1])], index = ['a','b','c'])


In [2]:
batch_size = 4

lr = 0.0000001

loss_fn = nn.CrossEntropyLoss()


In [3]:
class LSTM_predictor(nn.Module):
    def __init__(self, SIZE):
        super(LSTM_predictor, self).__init__()

        self.SIZE = SIZE

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(3, SIZE,2, dropout = 0.2)

        self.output = nn.Linear(SIZE, 2)

        self.activation = nn.Softmax(dim = 0)

        # self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, sentence):

        _, (lstm_out, _) = self.lstm(sentence)
        
        # print(lstm_out.shape)
        out = self.output(lstm_out[-1])
        # print(out.shape)
        # return self.softmax(out)


        # out = torch.sum(out,axis = 0)

        out = self.activation(out)

        return out
model = LSTM_predictor(16)

In [4]:
print(model)

LSTM_predictor(
  (lstm): LSTM(3, 16, num_layers=2, dropout=0.2)
  (output): Linear(in_features=16, out_features=2, bias=True)
  (activation): Softmax(dim=0)
)


In [5]:
def test(model, X_test, y_test, min_length):
    model.eval()
    test_batches = data_loader(X_test,y_test,batch_size=batch_size, min_length = min_length)
    acc = 0.0
    i = 0
    F1 = 0
    for data, labels in test_batches:
        pred = model(data).view((-1,2))
        a, b = torch.argmax(pred,1),torch.argmax(labels,1)
        corr = (a == b).float().mean().item()
        F1 = f1_score(a,b)
        acc += corr
        i += 1
    return acc/i, F1/i

def train(model,n_epochs,lr, X, y, X_test, y_test, min_length):
    parameters = model.parameters()

    optimizer = torch.optim.Adam(parameters, lr=lr)

    model.train()

    losss, accs, f1s = [], [], []
    pe = 10
    for i in range(n_epochs):
        model.train()
        train_batches = data_loader(X,y,batch_size=batch_size, min_length = min_length)

        sum_loss = 0.0
        i_loss = 0
        for data, labels in train_batches:
            pred = model(data)
            loss_fn = F.binary_cross_entropy
            loss = loss_fn(pred,labels)
            loss.backward()
            optimizer.step()

            sum_loss += loss.item()
            i_loss += 1
        
        if i%pe == 0:
            losss.append(sum_loss/i_loss)
            t, F1 = test(model,X_test, y_test, 70)
            accs.append(t)
            f1s.append(F1)

            print(f"epoch {i}/{n_epochs} - loss {sum_loss/i_loss:.3} - acc {t:.3} - F1 {F1:.3}")

    return np.array(losss), np.array(accs), np.array(f1s)

# X_encoded[0]

In [6]:
N = 4*int(1e4)

min_length = 50

# X_train, y_train = generate_data(0.8, train_test_validation="train", type = 2)

# X_test, y_test = generate_data(0, train_test_validation="test", type = 2)
X_validation, y_validation = generate_data(train_test_validation="validation", type = 2)

/kaggle/working/generate_data.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  true = np.array(make_true(train_test_validation, type))
/kaggle/working/generate_data.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  false = np.array(make_false(train_test_validation, type))
/kaggle/working/generate_data.py:113: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=objec

In [7]:
# train(model, n_epochs = 10)


In [8]:
def train_and_test(model, lr):
    N = 4*int(1e4)

    min_length = 20

    X_train, y_train = generate_data("train", type = 2)
    X_validation, y_validation = generate_data("validation", type = 2)
    loss, acc, f1s = train(model, 2000, lr, X_train, y_train, X_validation, y_validation, min_length = 20)
    
    min_length = 50

    X_test, y_test = generate_data("test", type = 2)

    t, f1 = test(model, X_test, y_test, min_length = 50)
    
    print(f"Final acc {t:.3} - F1 {f1}")

    return loss, acc, t, f1


lrs = [0.00001,0.00005, 0.0001]
sizes = [8, 16, 32]


save_i = 0
for lr in lrs:
    for size in sizes:
        losss, accs, faccs, ff1s = [], [], [], []
        data = {"lr":lr,'size':size}

        for i in range(3):
            model = LSTM_predictor(size)
            print("Training with lr", lr,"and size", size)
            loss, acc, facc, ff1 = train_and_test(model, lr)
            losss.append(loss)
            accs.append(acc)
            faccs.append(facc) 
            ff1s.append(ff1) 

        save_i += 1
        data['loss'] = np.array(losss)
        data['acc'] = np.array(accs)
        data['facc'] = np.array(faccs)
        data['ff1'] = np.array(ff1s)

        np.save(f"data{save_i}.npy",data, allow_pickle=True)

        



Training with lr 1e-05 and size 8


epoch 0/2000 - loss 0.837 - acc 0.35 - F1 0.1


epoch 10/2000 - loss 0.837 - acc 0.55 - F1 0.1


epoch 20/2000 - loss 0.838 - acc 0.45 - F1 0.133


epoch 30/2000 - loss 0.836 - acc 0.3 - F1 0.171


...

epoch 240/2000 - loss 0.835 - acc 0.4 - F1 0.08


epoch 250/2000 - loss 0.836 - acc 0.35 - F1 0.133


epoch 260/2000 - loss 0.832 - acc 0.5 - F1 0.08


epoch 270/2000 - loss 0.832 - acc 0.45 - F1 0.0


epoch 280/2000 - loss 0.836 - acc 0.45 - F1 0.133


In [ ]:
sum(range(21))